# Project: Identify Customer Segments

In this project, you will apply unsupervised learning techniques to identify segments of the population that form the core customer base for a mail-order sales company in Germany. These segments can then be used to direct marketing campaigns towards audiences that will have the highest expected rate of returns. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

This notebook will help you complete this task by providing a framework within which you will perform your analysis steps. In each step of the project, you will see some text describing the subtask that you will perform, followed by one or more code cells for you to complete your work. **Feel free to add additional code and markdown cells as you go along so that you can explore everything in precise chunks.** The code cells provided in the base template will outline only the major tasks, and will usually not be enough to cover all of the minor tasks that comprise it.

It should be noted that while there will be precise guidelines on how you should handle certain tasks in the project, there will also be places where an exact specification is not provided. **There will be times in the project where you will need to make and justify your own decisions on how to treat the data.** These are places where there may not be only one way to handle the data. In real-life tasks, there may be many valid ways to approach an analysis task. One of the most important things you can do is clearly document your approach so that other scientists can understand the decisions you've made.

At the end of most sections, there will be a Markdown cell labeled **Discussion**. In these cells, you will report your findings for the completed section, as well as document the decisions that you made in your approach to each subtask. **Your project will be evaluated not just on the code used to complete the tasks outlined, but also your communication about your observations and conclusions at each stage.**

In [ ]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# magic word for producing visualizations in notebook
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

'''
Import note: The classroom currently uses sklearn version 0.19.
If you need to use an imputer, it is available in sklearn.preprocessing.Imputer,
instead of sklearn.impute as in newer versions of sklearn.
'''

In [ ]:
# to print out all the outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
def get_values(df, columns):
    """
    Take a dataframe and a list of columns and
    returns the value counts for the columns.
    """
    for column in columns:
        print(column)
        print('=====================================')
        print(df[column].value_counts(dropna=False))
        print('\n')

def show_values(df, param):
    if param == 'all':
        get_values(df, df.columns)
    else:
        get_values(df, param) 

In [ ]:
def split_missing(missing_or_unknown):
    new_list=list()
    for values in missing_or_unknown:
        #print(values[1:-1])
        new_list.append(values[1:-1].split(","))
        #print(values)
    return new_list

### Step 0: Load the Data

There are four files associated with this project (not including this one):

- `Udacity_AZDIAS_Subset.csv`: Demographics data for the general population of Germany; 891211 persons (rows) x 85 features (columns).
- `Udacity_CUSTOMERS_Subset.csv`: Demographics data for customers of a mail-order company; 191652 persons (rows) x 85 features (columns).
- `Data_Dictionary.md`: Detailed information file about the features in the provided datasets.
- `AZDIAS_Feature_Summary.csv`: Summary of feature attributes for demographics data; 85 features (rows) x 4 columns

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. You will use this information to cluster the general population into groups with similar demographic properties. Then, you will see how the people in the customers dataset fit into those created clusters. The hope here is that certain clusters are over-represented in the customers data, as compared to the general population; those over-represented clusters will be assumed to be part of the core userbase. This information can then be used for further applications, such as targeting for a marketing campaign.

To start off with, load in the demographics data for the general population into a pandas DataFrame, and do the same for the feature attributes summary. Note for all of the `.csv` data files in this project: they're semicolon (`;`) delimited, so you'll need an additional argument in your [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call to read in the data properly. Also, considering the size of the main dataset, it may take some time for it to load completely.

Once the dataset is loaded, it's recommended that you take a little bit of time just browsing the general structure of the dataset and feature summary file. You'll be getting deep into the innards of the cleaning in the first major step of the project, so gaining some general familiarity can help you get your bearings.

In [ ]:
# Load in the general demographics data.
azdias = pd.read_csv('Udacity_AZDIAS_Subset.csv', sep=';')

# Load in the feature summary file.
feat_info = pd.read_csv('AZDIAS_Feature_Summary.csv', sep=';')

In [ ]:
# Check the structure of the data after it's loaded (e.g. print the number of
# rows and columns, print the first few rows).
azdias.shape
feat_info.shape

In [ ]:
azdias.head()
feat_info.head()

In [ ]:
summary = feat_info.copy()

In [ ]:
def show_missing(df):
    """
    Takes a dataframe and returns a dataframe with stats
    on missing and null values with their percentages.
    """
    null_count = df.isnull().sum()
    null_percentage = (null_count / df.shape[0]) * 100
    empty_count = pd.Series(((df == ' ') | (df == '')).sum())
    empty_percentage = (empty_count / df.shape[0]) * 100
    nan_count = pd.Series(((df == 'nan') | (df == 'NaN')).sum())
    nan_percentage = (nan_count / df.shape[0]) * 100
    dfx = pd.DataFrame({'num_missing': null_count, 'missing_percentage': null_percentage,
                         'num_empty': empty_count, 'empty_percentage': empty_percentage,
                         'nan_count': nan_count, 'nan_percentage': nan_percentage})
    return dfx

> **Tip**: Add additional cells to keep everything in reasonably-sized chunks! Keyboard shortcut `esc --> a` (press escape to enter command mode, then press the 'A' key) adds a new cell before the active cell, and `esc --> b` adds a new cell after the active cell. If you need to convert an active cell to a markdown cell, use `esc --> m` and to convert to a code cell, use `esc --> y`. 

## Step 1: Preprocessing

### Step 1.1: Assess Missing Data

The feature summary file contains a summary of properties for each demographics data column. You will use this file to help you make cleaning decisions during this stage of the project. First of all, you should assess the demographics data in terms of missing data. Pay attention to the following points as you perform your analysis, and take notes on what you observe. Make sure that you fill in the **Discussion** cell with your findings and decisions at the end of each step that has one!

#### Step 1.1.1: Convert Missing Value Codes to NaNs
The fourth column of the feature attributes summary (loaded in above as `feat_info`) documents the codes from the data dictionary that indicate missing or unknown data. While the file encodes this as a list (e.g. `[-1,0]`), this will get read in as a string object. You'll need to do a little bit of parsing to make use of it to identify and clean the data. Convert data that matches a 'missing' or 'unknown' value code into a numpy NaN value. You might want to see how much data takes on a 'missing' or 'unknown' code, and how much data is naturally missing, as a point of interest.

**As one more reminder, you are encouraged to add additional cells to break up your analysis into manageable chunks.**

In [ ]:
missing_values=split_missing(feat_info["missing_or_unknown"])

In [ ]:
# Identify missing or unknown data values and convert them to NaNs.
for attribute,missing_values_list in zip(feat_info["attribute"], missing_values):
    if missing_values_list[0] != "": # if the list not empty 
        for missing_value in missing_values_list:
            #check if the missing value is number (negative or positive numbers)
            if missing_value.isnumeric() or missing_value.lstrip('-').isnumeric():
                missing_value = int(missing_value)
            azdias.loc[azdias[attribute] == missing_value, attribute] = np.nan

#### Step 1.1.2: Assess Missing Data in Each Column

How much missing data is present in each column? There are a few columns that are outliers in terms of the proportion of values that are missing. You will want to use matplotlib's [`hist()`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.hist.html) function to visualize the distribution of missing value counts to find these columns. Identify and document these columns. While some of these columns might have justifications for keeping or re-encoding the data, for this project you should just remove them from the dataframe. (Feel free to make remarks about these outlier columns in the discussion, however!)

For the remaining features, are there any patterns in which columns have, or share, missing data?

In [ ]:
azdias_columns_with_nans = azdias.isnull().sum()[azdias.isnull().sum() > 0]

In [ ]:
azdias_nan_dictionary = {'sum': azdias_columns_with_nans.values, 'percent': np.round(azdias_columns_with_nans.values * 100 / azdias.shape[0], 2)}

In [ ]:
azdias_with_nans = pd.DataFrame(data=azdias_nan_dictionary, index=azdias_columns_with_nans.index)

In [ ]:
azdias_with_nans.sort_values(by="percent", ascending=False)

In [ ]:
# Investigate patterns in the amount of missing data in each column.
azdias_with_nans.sort_values(by="percent", ascending=False)[:50].plot.bar(figsize=(16,8))

In [ ]:
# Remove the outlier columns from the dataset. (You'll perform other data
# engineering tasks such as re-encoding and imputation later.)
columns_to_drop = azdias_with_nans[azdias_with_nans.percent > 33.].index.tolist()

In [ ]:
columns_to_drop

In [ ]:
azdias.drop(columns=columns_to_drop, inplace=True)
azdias.shape

In [ ]:
# Investigate patterns in the amount of missing data in each column.
show_missing(summary)

#### Discussion 1.1.2: Assess Missing Data in Each Column

- There is only column that has more than 33% null values. The "KK_KUNDENTYP" was dropped from the dataset.
- Most columns that has nulls are missing between 8% to 15%.
- Column "HH_EINKOMMEN_SCORE" stands out with one column missing 2% of its data.
- The rest, about ten (10) of them have less than 1% missing data. 

#### Step 1.1.3: Assess Missing Data in Each Row

Now, you'll perform a similar assessment for the rows of the dataset. How much data is missing in each row? As with the columns, you should see some groups of points that have a very different numbers of missing values. Divide the data into two subsets: one for data points that are above some threshold for missing values, and a second subset for points below that threshold.

In order to know what to do with the outlier rows, we should see if the distribution of data values on columns that are not missing data (or are missing very little data) are similar or different between the two groups. Select at least five of these columns and compare the distribution of values.
- You can use seaborn's [`countplot()`](https://seaborn.pydata.org/generated/seaborn.countplot.html) function to create a bar chart of code frequencies and matplotlib's [`subplot()`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.subplot.html) function to put bar charts for the two subplots side by side.
- To reduce repeated code, you might want to write a function that can perform this comparison, taking as one of its arguments a column to be compared.

Depending on what you observe in your comparison, this will have implications on how you approach your conclusions later in the analysis. If the distributions of non-missing features look similar between the data with many missing values and the data with few or no missing values, then we could argue that simply dropping those points from the analysis won't present a major issue. On the other hand, if the data with many missing values looks very different from the data with few or no missing values, then we should make a note on those data as special. We'll revisit these data later on. **Either way, you should continue your analysis for now using just the subset of the data with few or no missing values.**

In [ ]:
# How much data is missing in each row of the dataset?
azdias_rows_with_nans = azdias.isnull().sum(axis=1)
azdias_rows_with_nans.describe()

In [ ]:
len(azdias_rows_with_nans[azdias_rows_with_nans > 34])

In [ ]:
# Write code to divide the data into two subsets based on the number of missing
# values in each row.

min_count =  50
df = azdias.dropna(axis=0,
                   thresh=min_count)

In [ ]:
azdias.shape
df.shape

In [ ]:
azdias.columns

In [ ]:
# Compare the distribution of values for at least five columns where there are
# no or few missing values, between the two subsets.

df[['LP_FAMILIE_GROB', 'NATIONALITAET_KZ', 'SEMIO_FAM', 'VERS_TYP', 'BALLRAUM']].plot.hist(bins=12, alpha=0.5)

In [ ]:
azdias[['LP_FAMILIE_GROB', 'NATIONALITAET_KZ', 'SEMIO_FAM', 'VERS_TYP', 'BALLRAUM']].plot.hist(bins=12, alpha=0.5)

#### Discussion 1.1.3: Assess Missing Data in Each Row

Upon visual examination of that data, I believe that the two subsets of data were not qualitatively different from each other. As the histograms above shows, the distribution of the values for the random five columns are not that different.

### Step 1.2: Select and Re-Encode Features

Checking for missing data isn't the only way in which you can prepare a dataset for analysis. Since the unsupervised learning techniques to be used will only work on data that is encoded numerically, you need to make a few encoding changes or additional assumptions to be able to make progress. In addition, while almost all of the values in the dataset are encoded using numbers, not all of them represent numeric values. Check the third column of the feature summary (`feat_info`) for a summary of types of measurement.
- For numeric and interval data, these features can be kept without changes.
- Most of the variables in the dataset are ordinal in nature. While ordinal values may technically be non-linear in spacing, make the simplifying assumption that the ordinal variables can be treated as being interval in nature (that is, kept without any changes).
- Special handling may be necessary for the remaining two variable types: categorical, and 'mixed'.

In the first two parts of this sub-step, you will perform an investigation of the categorical and mixed-type features and make a decision on each of them, whether you will keep, drop, or re-encode each. Then, in the last part, you will create a new data frame with only the selected and engineered columns.

Data wrangling is often the trickiest part of the data analysis process, and there's a lot of it to be done here. But stick with it: once you're done with this step, you'll be ready to get to the machine learning parts of the project!

In [ ]:
# How many features are there of each data type?

summary.type.value_counts()

#### Step 1.2.1: Re-Encode Categorical Features

For categorical data, you would ordinarily need to encode the levels as dummy variables. Depending on the number of categories, perform one of the following:
- For binary (two-level) categoricals that take numeric values, you can keep them without needing to do anything.
- There is one binary variable that takes on non-numeric values. For this one, you need to re-encode the values as numbers or create a dummy variable.
- For multi-level categoricals (three or more values), you can choose to encode the values using multiple dummy variables (e.g. via [OneHotEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)), or (to keep things straightforward) just drop them from the analysis. As always, document your choices in the Discussion section.

In [ ]:
# Assess categorical variables: which are binary, which are multi-level, and
# which one needs to be re-encoded?

categorical_features = summary[summary.type == 'categorical'].attribute
categorical_features

In [ ]:
categorical_features = categorical_features.drop([0,40, 47])
categorical_features

In [ ]:
# Split categorical variables into binary or multi buckets
categorical_binary = []
categorical_multi = []
for feature in categorical_features:
    if azdias[feature].nunique() > 2:
        categorical_multi.append(feature)
    else:
        categorical_binary.append(feature)

In [ ]:
show_values(df, categorical_binary)

In [ ]:
# Standardize columns into 0's or 1's
df['ANREDE_KZ'].replace([2, 1], [1, 0], inplace=True)
df['GREEN_AVANTGARDE'].replace([0, 1], [1, 0], inplace=True)
df['SOHO_KZ'].replace([0.0, 1.0], [1, 0], inplace=True)
df['OST_WEST_KZ'].replace(['W', 'O'], [1, 0], inplace=True)

In [ ]:
show_values(df, categorical_binary)

In [ ]:
# Re-encode categorical variable(s) to be kept in the analysis.
show_values(df, categorical_multi)

In [ ]:
# One Hot Encoding and remove reference columns
df = pd.get_dummies(df, columns=categorical_multi)

#### Discussion 1.2.1: Re-Encode Categorical Features

- I standadized the categorical columns that are binary into 0's and 1's
- I performed One Hot Encoding for those columns that are not binary by using the get_dummies function.

#### Step 1.2.2: Engineer Mixed-Type Features

There are a handful of features that are marked as "mixed" in the feature summary that require special treatment in order to be included in the analysis. There are two in particular that deserve attention; the handling of the rest are up to your own choices:
- "PRAEGENDE_JUGENDJAHRE" combines information on three dimensions: generation by decade, movement (mainstream vs. avantgarde), and nation (east vs. west). While there aren't enough levels to disentangle east from west, you should create two new variables to capture the other two dimensions: an interval-type variable for decade, and a binary variable for movement.
- "CAMEO_INTL_2015" combines information on two axes: wealth and life stage. Break up the two-digit codes by their 'tens'-place and 'ones'-place digits into two new ordinal variables (which, for the purposes of this project, is equivalent to just treating them as their raw numeric values).
- If you decide to keep or engineer new features around the other mixed-type features, make sure you note your steps in the Discussion section.

Be sure to check `Data_Dictionary.md` for the details needed to finish these tasks.

In [ ]:
# Investigate "PRAEGENDE_JUGENDJAHRE" and engineer two new variables.

show_values(df, ['PRAEGENDE_JUGENDJAHRE'])

### 1.18. PRAEGENDE_JUGENDJAHRE
Dominating movement of person's youth (avantgarde vs. mainstream; east vs. west)
- -1: unknown
-  0: unknown
-  1: 40s - war years (Mainstream, E+W)
-  2: 40s - reconstruction years (Avantgarde, E+W)
-  3: 50s - economic miracle (Mainstream, E+W)
-  4: 50s - milk bar / Individualisation (Avantgarde, E+W)
-  5: 60s - economic miracle (Mainstream, E+W)
-  6: 60s - generation 68 / student protestors (Avantgarde, W)
-  7: 60s - opponents to the building of the Wall (Avantgarde, E)
-  8: 70s - family orientation (Mainstream, E+W)
-  9: 70s - peace movement (Avantgarde, E+W)
- 10: 80s - Generation Golf (Mainstream, W)
- 11: 80s - ecological awareness (Avantgarde, W)
- 12: 80s - FDJ / communist party youth organisation (Mainstream, E)
- 13: 80s - Swords into ploughshares (Avantgarde, E)
- 14: 90s - digital media kids (Mainstream, E+W)
- 15: 90s - ecological awareness (Avantgarde, E+W)

In [ ]:
df['PJ_40S'] = np.where(df['PRAEGENDE_JUGENDJAHRE'].isin([1,2]), 1, 0)
df['PJ_50S'] = np.where(df['PRAEGENDE_JUGENDJAHRE'].isin([3,4]), 1, 0)
df['PJ_60S'] = np.where(df['PRAEGENDE_JUGENDJAHRE'].isin([5,6,7]), 1, 0)
df['PJ_70S'] = np.where(df['PRAEGENDE_JUGENDJAHRE'].isin([8,9]), 1, 0)
df['PJ_80S'] = np.where(df['PRAEGENDE_JUGENDJAHRE'].isin([10,11,12,13]), 1, 0)
df['PJ_90S'] = np.where(df['PRAEGENDE_JUGENDJAHRE'].isin([14,15]), 1, 0)

In [ ]:
df.head()

In [ ]:
df.drop(columns=['PRAEGENDE_JUGENDJAHRE'], inplace=True)

In [ ]:
# Investigate "CAMEO_INTL_2015" and engineer two new variables.

show_values(df, ['CAMEO_INTL_2015'])

### 4.3. CAMEO_INTL_2015
German CAMEO: Wealth / Life Stage Typology, mapped to international code
- -1: unknown
- 11: Wealthy Households - Pre-Family Couples & Singles
- 12: Wealthy Households - Young Couples With Children
- 13: Wealthy Households - Families With School Age Children
- 14: Wealthy Households - Older Families &  Mature Couples
- 15: Wealthy Households - Elders In Retirement
- 21: Prosperous Households - Pre-Family Couples & Singles
- 22: Prosperous Households - Young Couples With Children
- 23: Prosperous Households - Families With School Age Children
- 24: Prosperous Households - Older Families & Mature Couples
- 25: Prosperous Households - Elders In Retirement
- 31: Comfortable Households - Pre-Family Couples & Singles
- 32: Comfortable Households - Young Couples With Children
- 33: Comfortable Households - Families With School Age Children
- 34: Comfortable Households - Older Families & Mature Couples
- 35: Comfortable Households - Elders In Retirement
- 41: Less Affluent Households - Pre-Family Couples & Singles
- 42: Less Affluent Households - Young Couples With Children
- 43: Less Affluent Households - Families With School Age Children
- 44: Less Affluent Households - Older Families & Mature Couples
- 45: Less Affluent Households - Elders In Retirement
- 51: Poorer Households - Pre-Family Couples & Singles
- 52: Poorer Households - Young Couples With Children
- 53: Poorer Households - Families With School Age Children
- 54: Poorer Households - Older Families & Mature Couples
- 55: Poorer Households - Elders In Retirement
- XX: unknown

In [ ]:
df['CI_wealthy'] = np.where(df['CAMEO_INTL_2015'].isin([11,12,13,14,15]), 1, 0)
df['CI_prosperous'] = np.where(df['CAMEO_INTL_2015'].isin([21,22,23,24,25]), 1, 0)
df['CI_comfortableS'] = np.where(df['CAMEO_INTL_2015'].isin([31,32,33,34,35]), 1, 0)
df['CI_less'] = np.where(df['CAMEO_INTL_2015'].isin([41,42,43,44,45]), 1, 0)
df['CI_poor'] = np.where(df['CAMEO_INTL_2015'].isin([51,52,53,54,55]), 1, 0)

In [ ]:
df.drop(columns=['CAMEO_INTL_2015'], inplace=True)

#### Discussion 1.2.2: Engineer Mixed-Type Features

Using the data dictionary as a reference, I engineered 11 new features and drop the two original columns.

#### Step 1.2.3: Complete Feature Selection

In order to finish this step up, you need to make sure that your data frame now only has the columns that you want to keep. To summarize, the dataframe should consist of the following:
- All numeric, interval, and ordinal type columns from the original dataset.
- Binary categorical features (all numerically-encoded).
- Engineered features from other multi-level categorical features and mixed features.

Make sure that for any new columns that you have engineered, that you've excluded the original columns from the final dataset. Otherwise, their values will interfere with the analysis later on the project. For example, you should not keep "PRAEGENDE_JUGENDJAHRE", since its values won't be useful for the algorithm: only the values derived from it in the engineered features you created should be retained. As a reminder, your data should only be from **the subset with few or no missing values**.

In [ ]:
# If there are other re-engineering tasks you need to perform, make sure you
# take care of them here. (Dealing with missing data will come in step 2.1.)

df.shape

In [ ]:
# Do whatever you need to in order to ensure that the dataframe only contains
# the columns that should be passed to the algorithm functions.

df.dropna(inplace=True)
df.shape

### Step 1.3: Create a Cleaning Function

Even though you've finished cleaning up the general population demographics data, it's important to look ahead to the future and realize that you'll need to perform the same cleaning steps on the customer demographics data. In this substep, complete the function below to execute the main feature selection, encoding, and re-engineering steps you performed above. Then, when it comes to looking at the customer data in Step 3, you can just run this function on that DataFrame to get the trimmed dataset in a single step.

In [ ]:
def clean_dataframe(df):
    # Identify missing or unknown data values and convert them to NaNs.
    for attribute,missing_values_list in zip(feat_info["attribute"], missing_values):
        if missing_values_list[0] != "": # if the list not empty 
            for missing_value in missing_values_list:
                #check if the missing value is number (negative or positive numbers)
                if missing_value.isnumeric() or missing_value.lstrip('-').isnumeric():
                    missing_value = int(missing_value)
                df.loc[azdias[attribute] == missing_value, attribute] = np.nan
    df_with_nans = df.isnull().sum()[df.isnull().sum() > 0]
    df_nan_dictionary = {'sum': df_with_nans.values, 'percent': np.round(df_with_nans.values * 100 / azdias.shape[0], 2)}
    df_with_nans = pd.DataFrame(data=df_nan_dictionary, index=df_with_nans.index)
    columns_to_drop = df_with_nans[df_with_nans.percent > 33.].index.tolist()
    df.drop(columns=columns_to_drop, inplace=True)
    df_rows_with_nans = df.isnull().sum(axis=1)
    df = df.dropna(axis=0,
                   thresh=47)
    categorical_features = summary[summary.type == 'categorical'].attribute
    categorical_features = categorical_features.drop([0,40, 47])
    categorical_binary = []
    categorical_multi = []
    for feature in categorical_features:
        if df[feature].nunique() > 2:
            categorical_multi.append(feature)
        else:
            categorical_binary.append(feature)
    df['ANREDE_KZ'].replace([2, 1], [1, 0], inplace=True)
    df['GREEN_AVANTGARDE'].replace([0, 1], [1, 0], inplace=True)
    df['SOHO_KZ'].replace([0.0, 1.0], [1, 0], inplace=True)
    df['OST_WEST_KZ'].replace(['W', 'O'], [1, 0], inplace=True)
    df = pd.get_dummies(df, columns=categorical_multi)
    df['PJ_40S'] = np.where(df['PRAEGENDE_JUGENDJAHRE'].isin([1,2]), 1, 0)
    df['PJ_50S'] = np.where(df['PRAEGENDE_JUGENDJAHRE'].isin([3,4]), 1, 0)
    df['PJ_60S'] = np.where(df['PRAEGENDE_JUGENDJAHRE'].isin([5,6,7]), 1, 0)
    df['PJ_70S'] = np.where(df['PRAEGENDE_JUGENDJAHRE'].isin([8,9]), 1, 0)
    df['PJ_80S'] = np.where(df['PRAEGENDE_JUGENDJAHRE'].isin([10,11,12,13]), 1, 0)
    df['PJ_90S'] = np.where(df['PRAEGENDE_JUGENDJAHRE'].isin([14,15]), 1, 0)
    df.drop(columns=['PRAEGENDE_JUGENDJAHRE'], inplace=True)
    df['CI_wealthy'] = np.where(df['CAMEO_INTL_2015'].isin([11,12,13,14,15]), 1, 0)
    df['CI_prosperous'] = np.where(df['CAMEO_INTL_2015'].isin([21,22,23,24,25]), 1, 0)
    df['CI_comfortableS'] = np.where(df['CAMEO_INTL_2015'].isin([31,32,33,34,35]), 1, 0)
    df['CI_less'] = np.where(df['CAMEO_INTL_2015'].isin([41,42,43,44,45]), 1, 0)
    df['CI_poor'] = np.where(df['CAMEO_INTL_2015'].isin([51,52,53,54,55]), 1, 0)
    df.drop(columns=['CAMEO_INTL_2015'], inplace=True)
    df.dropna(inplace=True)
    df.reset_index().drop(columns=['index'], inplace=True)
    return df  

## Step 2: Feature Transformation

### Step 2.1: Apply Feature Scaling

Before we apply dimensionality reduction techniques to the data, we need to perform feature scaling so that the principal component vectors are not influenced by the natural differences in scale for features. Starting from this part of the project, you'll want to keep an eye on the [API reference page for sklearn](http://scikit-learn.org/stable/modules/classes.html) to help you navigate to all of the classes and functions that you'll need. In this substep, you'll need to check the following:

- sklearn requires that data not have missing values in order for its estimators to work properly. So, before applying the scaler to your data, make sure that you've cleaned the DataFrame of the remaining missing values. This can be as simple as just removing all data points with missing data, or applying an [Imputer](https://scikit-learn.org/0.16/modules/generated/sklearn.preprocessing.Imputer.html) to replace all missing values. You might also try a more complicated procedure where you temporarily remove missing values in order to compute the scaling parameters before re-introducing those missing values and applying imputation. Think about how much missing data you have and what possible effects each approach might have on your analysis, and justify your decision in the discussion section below.
- For the actual scaling function, a [StandardScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) instance is suggested, scaling each feature to mean 0 and standard deviation 1.
- For these classes, you can make use of the `.fit_transform()` method to both fit a procedure to the data as well as apply the transformation to the data at the same time. Don't forget to keep the fit sklearn objects handy, since you'll be applying them to the customer demographics data towards the end of the project.

In [ ]:
# If you've not yet cleaned the dataset of all NaN values, then investigate and
# do that now.

df.info()

In [ ]:
show_missing(df)

In [ ]:
# Apply feature scaling to the general population demographics data.

scaler = StandardScaler()
df_std = scaler.fit_transform(df)

### Discussion 2.1: Apply Feature Scaling

Rather than messing with imputation for the missing data or nulls, I decided to just trim the whole with .dropna() because the difference in the number rows is not really that much.

### Step 2.2: Perform Dimensionality Reduction

On your scaled data, you are now ready to apply dimensionality reduction techniques.

- Use sklearn's [PCA](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) class to apply principal component analysis on the data, thus finding the vectors of maximal variance in the data. To start, you should not set any parameters (so all components are computed) or set a number of components that is at least half the number of features (so there's enough features to see the general trend in variability).
- Check out the ratio of variance explained by each principal component as well as the cumulative variance explained. Try plotting the cumulative or sequential values using matplotlib's [`plot()`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.plot.html) function. Based on what you find, select a value for the number of transformed features you'll retain for the clustering part of the project.
- Once you've made a choice for the number of components to keep, make sure you re-fit a PCA instance to perform the decided-on transformation.

In [ ]:
# Apply PCA to the data.

pca = PCA()
df_pca = pca.fit_transform(df_std)

In [ ]:
# Investigate the variance accounted for by each principal component.

# Taken from Udacity course - Supervised and  Unsupervised Learning
def scree_plot(pca, components_no=None, show_labels=False):

    vals = pca.explained_variance_ratio_
    if components_no:
        vals = vals[:components_no]
    num_components = len(vals)
    ind = np.arange(num_components)
 
    plt.figure(figsize=(18, 8))
    ax = plt.subplot(111)
    cumvals = np.cumsum(vals)
    ax.bar(ind, vals)
    ax.plot(ind, cumvals)
    
    if show_labels:
        for i in range(num_components):
            ax.annotate(r"%s%%" % ((str(vals[i]*100)[:4])), (ind[i]+0.2, vals[i]), va="bottom", ha="center", fontsize=12)
 
    ax.xaxis.set_tick_params(width=0)
    ax.yaxis.set_tick_params(width=2, length=12)
 
    ax.set_xlabel("Principal Component")
    ax.set_ylabel("Variance Explained (%)")
    plt.title('Explained Variance Per Principal Component')

In [ ]:
scree_plot(pca, show_labels=False)

In [ ]:
scree_plot(pca, components_no=80, show_labels=False)

In [ ]:
pca.explained_variance_ratio_[:5].sum()

 The first 5 components capture more than 20% of the variance.

In [ ]:
for i in np.arange(10, 151, 10):
    print('{} components explain {} of variance.'.format(i, pca.explained_variance_ratio_[:i].sum()))

We'll pick 120 here for 90% variability.

In [ ]:
df_std.shape

In [ ]:
# Re-apply PCA to the data while selecting for number of components to retain.

pca = PCA(n_components=120)
df_pca = pca.fit_transform(df_std)

### Discussion 2.2: Perform Dimensionality Reduction

With PCA, I was able to reduce the number of features from 225 to 120. This explains about 90% variability.

### Step 2.3: Interpret Principal Components

Now that we have our transformed principal components, it's a nice idea to check out the weight of each variable on the first few components to see if they can be interpreted in some fashion.

As a reminder, each principal component is a unit vector that points in the direction of highest variance (after accounting for the variance captured by earlier principal components). The further a weight is from zero, the more the principal component is in the direction of the corresponding feature. If two features have large weights of the same sign (both positive or both negative), then increases in one tend expect to be associated with increases in the other. To contrast, features with different signs can be expected to show a negative correlation: increases in one variable should result in a decrease in the other.

- To investigate the features, you should map each weight to their corresponding feature name, then sort the features according to weight. The most interesting features for each principal component, then, will be those at the beginning and end of the sorted list. Use the data dictionary document to help you understand these most prominent features, their relationships, and what a positive or negative value on the principal component might indicate.
- You should investigate and interpret feature associations from the first three principal components in this substep. To help facilitate this, you should write a function that you can call at any time to print the sorted list of feature weights, for the *i*-th principal component. This might come in handy in the next step of the project, when you interpret the tendencies of the discovered clusters.

In [ ]:
# Plot features_no features with the most positive and negative variance
# for a component_no PCA component
def pca_results(df, pca, component_no, features_no, show_plot=True):
    
    # Build a dataframe with features_no features capturing most variability
    # for a given component number (component_no)
    pca_comp = pd.DataFrame(np.round(pca.components_, 4), columns=df.keys()).iloc[component_no - 1]
    pca_comp.sort_values(ascending=False, inplace=True)
    pca_comp = pd.concat([pca_comp.head(features_no), pca_comp.tail(features_no)])
    
    if show_plot:
        # Plot the result
        pca_comp.plot(kind='bar', 
                  title='Most {} Weighted Features for PCA Component {}'.format(features_no*2, component_no),
                  figsize=(12, 6))
        plt.show()
    
    return pca_comp

In [ ]:
# Check first 3 components
for i in np.arange(0, 3, 1):
    res = pca_results(df, pca, i, 3)
    print(res)

### Discussion 2.3: Interpret Principal Components

First Component
- Positive Association
    - AGER_TYP_3
    - ZABEOTYP_2
    - GFK_URLAUBERTYP_9.0
- Negative Association
    - CAMEO_DEU_2015_5D
    - PJ_70S
    - MIN_GEBAEUDEJAHR    - 
Second Component
- Positive Association
    - LP_STATUS_GROB_1.0    
    - PLZ8_ANTG3            
    - HH_EINKOMMEN_SCORE    
- Negative Association
    - KBA05_ANTG1          
    - FINANZ_MINIMALIST   
    - MOBI_REGIO             
Third Component
- Positive Association
    - FINANZ_VORSORGER         
    - ALTERSKATEGORIE_GROB     
    - ZABEOTYP_3               
- Negative Association
    - FINANZ_ANLEGER
    - FINANZ_UNAUFFAELLIGER   
    - FINANZ_SPARER           

## Step 3: Clustering

### Step 3.1: Apply Clustering to General Population

You've assessed and cleaned the demographics data, then scaled and transformed them. Now, it's time to see how the data clusters in the principal components space. In this substep, you will apply k-means clustering to the dataset and use the average within-cluster distances from each point to their assigned cluster's centroid to decide on a number of clusters to keep.

- Use sklearn's [KMeans](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans) class to perform k-means clustering on the PCA-transformed data.
- Then, compute the average difference from each point to its assigned cluster's center. **Hint**: The KMeans object's `.score()` method might be useful here, but note that in sklearn, scores tend to be defined so that larger is better. Try applying it to a small, toy dataset, or use an internet search to help your understanding.
- Perform the above two steps for a number of different cluster counts. You can then see how the average distance decreases with an increasing number of clusters. However, each additional cluster provides a smaller net benefit. Use this fact to select a final number of clusters in which to group the data. **Warning**: because of the large size of the dataset, it can take a long time for the algorithm to resolve. The more clusters to fit, the longer the algorithm will take. You should test for cluster counts through at least 10 clusters to get the full picture, but you shouldn't need to test for a number of clusters above about 30.
- Once you've selected a final number of clusters to use, re-fit a KMeans instance to perform the clustering operation. Make sure that you also obtain the cluster assignments for the general demographics data, since you'll be using them in the final Step 3.3.

In [ ]:
# Create a sample dataset to reduce computational time
# Reduce the size to 10%
df_pca_sample = df_pca[np.random.choice(df_pca.shape[0], int(df_pca.shape[0]*0.1), replace=False)]
df_pca_sample 

In [ ]:
# Sum of Squared Errors
sse = []
   
ks = np.arange(20, 31)

# Over a number of different cluster counts...
for k in ks:
    # run k-means clustering on the data and...
    kmeans = KMeans(k, random_state=493, max_iter=30, n_jobs=-1).fit(df_pca_sample)
    score = np.abs(kmeans.score(df_pca_sample))
    sse.append(score)
    # compute the average within-cluster distances.
    print('Clustering done for {} k, with SSE {}'.format(k, score))    

In [ ]:
# Investigate the change in within-cluster distance across number of clusters.
# HINT: Use matplotlib's plot function to visualize this relationship.

# Use the Elbow Method to identify the right value of k
plt.figure(figsize=(16, 8))
plt.xticks(np.arange(0, ks[-1]+1, step=1))
plt.xlabel('K')
plt.ylabel('SSE')
plt.title('SSE vs. K')
plt.plot(ks, sse, linestyle='-', marker='o');

In [ ]:
# Re-fit the k-means model with the selected number of clusters and obtain
# cluster predictions for the general population demographics data.

k = 23
kmeans = KMeans(k, random_state=493, max_iter=30, n_jobs=-1).fit(df_pca)
population_clusters = kmeans.predict(df_pca)

### Discussion 3.1: Apply Clustering to General Population

The SSE v. k plot doesn't really look like an elbow but more like just a straight descending line. Hence, I opted for the k of 23 since it is in the middle.

### Step 3.2: Apply All Steps to the Customer Data

Now that you have clusters and cluster centers for the general population, it's time to see how the customer data maps on to those clusters. Take care to not confuse this for re-fitting all of the models to the customer data. Instead, you're going to use the fits from the general population to clean, transform, and cluster the customer data. In the last step of the project, you will interpret how the general population fits apply to the customer data.

- Don't forget when loading in the customers data, that it is semicolon (`;`) delimited.
- Apply the same feature wrangling, selection, and engineering steps to the customer demographics using the `clean_data()` function you created earlier. (You can assume that the customer demographics data has similar meaning behind missing data patterns as the general demographics data.)
- Use the sklearn objects from the general demographics data, and apply their transformations to the customers data. That is, you should not be using a `.fit()` or `.fit_transform()` method to re-fit the old objects, nor should you be creating new sklearn objects! Carry the data through the feature scaling, PCA, and clustering steps, obtaining cluster assignments for all of the data in the customer demographics data.

In [ ]:
# Load in the customer demographics data.
customers = pd.read_csv('Udacity_CUSTOMERS_Subset.csv', sep=';')

In [ ]:
# Apply preprocessing, feature transformation, and clustering from the general
# demographics onto the customer data, obtaining cluster predictions for the
# customer demographics data.
customers_clean = clean_dataframe(customers)

In [ ]:
customers_clean.shape

In [ ]:
# Remove rows with NaNs
customers_clean = customers_clean.dropna()
customers_clean.shape

In [ ]:
# Apply feature scaling to the customers demographics data
customers_clean_std = scaler.transform(customers_clean)

In [ ]:
customers_clean_std.head()

In [ ]:
# Apply PCA to the data
customers_pca = pca.transform(customers_clean_std)

### Step 3.3: Compare Customer Data to Demographics Data

At this point, you have clustered data based on demographics of the general population of Germany, and seen how the customer data for a mail-order sales company maps onto those demographic clusters. In this final substep, you will compare the two cluster distributions to see where the strongest customer base for the company is.

Consider the proportion of persons in each cluster for the general population, and the proportions for the customers. If we think the company's customer base to be universal, then the cluster assignment proportions should be fairly similar between the two. If there are only particular segments of the population that are interested in the company's products, then we should see a mismatch from one to the other. If there is a higher proportion of persons in a cluster for the customer data compared to the general population (e.g. 5% of persons are assigned to a cluster for the general population, but 15% of the customer data is closest to that cluster's centroid) then that suggests the people in that cluster to be a target audience for the company. On the other hand, the proportion of the data in a cluster being larger in the general population than the customer data (e.g. only 2% of customers closest to a population centroid that captures 6% of the data) suggests that group of persons to be outside of the target demographics.

Take a look at the following points in this step:

- Compute the proportion of data points in each cluster for the general population and the customer data. Visualizations will be useful here: both for the individual dataset proportions, but also to visualize the ratios in cluster representation between groups. Seaborn's [`countplot()`](https://seaborn.pydata.org/generated/seaborn.countplot.html) or [`barplot()`](https://seaborn.pydata.org/generated/seaborn.barplot.html) function could be handy.
  - Recall the analysis you performed in step 1.1.3 of the project, where you separated out certain data points from the dataset if they had more than a specified threshold of missing values. If you found that this group was qualitatively different from the main bulk of the data, you should treat this as an additional data cluster in this analysis. Make sure that you account for the number of data points in this subset, for both the general population and customer datasets, when making your computations!
- Which cluster or clusters are overrepresented in the customer dataset compared to the general population? Select at least one such cluster and infer what kind of people might be represented by that cluster. Use the principal component interpretations from step 2.3 or look at additional components to help you make this inference. Alternatively, you can use the `.inverse_transform()` method of the PCA and StandardScaler objects to transform centroids back to the original data space and interpret the retrieved values directly.
- Perform a similar investigation for the underrepresented clusters. Which cluster or clusters are underrepresented in the customer dataset compared to the general population, and what kinds of people are typified by these clusters?

In [ ]:
# Compare the proportion of data in each cluster for the customer data to the
# proportion of data in each cluster for the general population.

population_clusters = pd.Series(population_clusters)
population_cluster = population_clusters.value_counts().sort_index()
population_cluster = pd.Series(population_cluster)

customer_clusters = pd.Series(customer_clusters)
customer_cluster = customer_clusters.value_counts().sort_index()
customer_cluster = pd.Series(customer_cluster)

In [ ]:
# What kinds of people are part of a cluster that is overrepresented in the
# customer data compared to the general population?



In [ ]:
# What kinds of people are part of a cluster that is underrepresented in the
# customer data compared to the general population?



### Discussion 3.3: Compare Customer Data to Demographics Data

(Double-click this cell and replace this text with your own text, reporting findings and conclusions from the clustering analysis. Can we describe segments of the population that are relatively popular with the mail-order company, or relatively unpopular with the company?)

> Congratulations on making it this far in the project! Before you finish, make sure to check through the entire notebook from top to bottom to make sure that your analysis follows a logical flow and all of your findings are documented in **Discussion** cells. Once you've checked over all of your work, you should export the notebook as an HTML document to submit for evaluation. You can do this from the menu, navigating to **File -> Download as -> HTML (.html)**. You will submit both that document and this notebook for your project submission.